#Quantowerdash


In [16]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import webbrowser
import os

def detect_date_column(df):
    """Detect the date/time column from the DataFrame"""
    date_columns = df.select_dtypes(include=['datetime64']).columns
    if len(date_columns) > 0:
        return date_columns[0]
    
    for col in df.columns:
        if 'date' in col.lower() or 'time' in col.lower():
            try:
                pd.to_datetime(df[col])
                return col
            except:
                continue
    return None

def calculate_metrics(df, date_col):
    df[date_col] = pd.to_datetime(df[date_col])
    
    total_trades = len(df)
    winning_trades = len(df[df['Net P/L'] > 0])
    win_rate = (winning_trades / total_trades) * 100
    
    df['daily_returns'] = df.groupby(df[date_col].dt.date)['Net P/L'].transform('sum')
    risk_free_rate = 0.02
    daily_rf_rate = (1 + risk_free_rate) ** (1/252) - 1
    
    returns_std = df['daily_returns'].std()
    downside_returns = df['daily_returns'][df['daily_returns'] < 0]
    downside_std = downside_returns.std() if len(downside_returns) > 0 else 0
    
    avg_return = df['daily_returns'].mean()
    sharpe = (avg_return - daily_rf_rate) / returns_std if returns_std != 0 else 0
    sortino = (avg_return - daily_rf_rate) / downside_std if downside_std != 0 else 0
    
    return {
        'total_trades': total_trades,
        'win_rate': win_rate,
        'total_pnl': df['Net P/L'].sum(),
        'avg_profit': df['Net P/L'].mean(),
        'sharpe': sharpe,
        'sortino': sortino,
        'max_drawdown': df['Net P/L'].min(),
        'best_trade': df['Net P/L'].max(),
        'worst_trade': df['Net P/L'].min()
    }

def format_returns(x, pos):
    return f'{x:.1f}%'

def create_plots(df, date_col):
    # Define green color palette
    green_palette = ['#E8F5E9', '#C8E6C9', '#A5D6A7', '#81C784', '#66BB6A', 
                    '#4CAF50', '#43A047', '#388E3C', '#2E7D32', '#1B5E20']
    sns.set_palette(green_palette)
    
    # Main Performance Dashboard
    plt.figure(figsize=(15, 30))
    
    # Cumulative P&L
    plt.subplot(6, 1, 1)
    cum_pnl = df['Net P/L'].cumsum()
    sns.lineplot(data=cum_pnl, color='#2E7D32')
    plt.fill_between(range(len(cum_pnl)), cum_pnl, alpha=0.3, color='#81C784')
    plt.title('Cumulative P&L Over Time', color='#1B5E20', pad=20)
    
    # Returns Performance
    plt.subplot(6, 1, 2)
    df['returns_pct'] = (df['Net P/L'] / df['Trade value']) * 100
    total_return = df['returns_pct'].sum()
    
    sns.lineplot(data=df, x=date_col, y='returns_pct', color='#2E7D32')
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(format_returns))
    plt.title('Returns Performance', color='#1B5E20', pad=20)
    plt.xticks(rotation=45)
    
    color = '#2E7D32' if total_return >= 0 else '#B71C1C'
    plt.text(0.02, 0.95, f'Total Return: {total_return:.1f}%',
             transform=plt.gca().transAxes,
             fontsize=14,
             color=color,
             bbox=dict(facecolor='white', alpha=0.8, edgecolor=color))
    
    # P&L Distribution
    plt.subplot(6, 1, 3)
    sns.histplot(data=df, x='Net P/L', bins=50, color='#43A047')
    plt.title('P&L Distribution', color='#1B5E20', pad=20)
    
    # Trade Heatmap
    plt.subplot(6, 1, 4)
    df['DayOfWeek'] = df[date_col].dt.day_name()
    df['Hour'] = df[date_col].dt.hour
    pivot_table = pd.pivot_table(df, values='Net P/L', index='DayOfWeek', 
                               columns='Hour', aggfunc='mean')
    sns.heatmap(pivot_table, 
                cmap='Greens',
                center=0,
                annot=True,
                fmt='.0f')
    plt.title('Average P&L by Day and Hour', color='#1B5E20', pad=20)
    
    # Symbol Performance
    plt.subplot(6, 1, 5)
    symbol_pnl = df.groupby('Symbol')['Net P/L'].sum().sort_values()
    sns.barplot(x=symbol_pnl.index, y=symbol_pnl.values, palette='Greens')
    plt.xticks(rotation=45)
    plt.title('P&L by Symbol', color='#1B5E20', pad=20)
    
    # Best/Worst Analysis
    plt.subplot(6, 1, 6)
    daily_pnl = df.groupby('DayOfWeek')['Net P/L'].mean().sort_values()
    sns.barplot(x=daily_pnl.index, y=daily_pnl.values, palette='Greens')
    plt.title('Average P&L by Day', color='#1B5E20', pad=20)
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig('trading_analytics.png')
    plt.close()
    
    # Best/Worst Details Plot
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    best_trade = df.loc[df['Net P/L'].idxmax()]
    worst_trade = df.loc[df['Net P/L'].idxmin()]
    best_day = daily_pnl.idxmax()
    worst_day = daily_pnl.idxmin()
    
    # Best/Worst Days
    sns.barplot(x=daily_pnl.index, y=daily_pnl.values, ax=ax1, palette='Greens')
    ax1.set_title('P&L by Day of Week', color='#1B5E20')
    ax1.tick_params(axis='x', rotation=45)
    
    # Best Day Details
    ax2.axis('off')
    detail_color = '#2E7D32'
    ax2.text(0.1, 0.8, f"Best Trading Day: {best_day}", fontsize=12, 
             color=detail_color, fontweight='bold')
    ax2.text(0.1, 0.6, f"Average P&L: ${daily_pnl[best_day]:.2f}", 
             fontsize=12, color=detail_color)
    ax2.text(0.1, 0.4, f"Best Trade: ${best_trade['Net P/L']:.2f}", 
             fontsize=12, color=detail_color)
    ax2.text(0.1, 0.2, f"Time: {best_trade[date_col].strftime('%H:%M')}", 
             fontsize=12, color=detail_color)
    
    # Time Distribution
    hours_pnl = df.groupby(df[date_col].dt.hour)['Net P/L'].mean()
    sns.barplot(x=hours_pnl.index, y=hours_pnl.values, ax=ax3, palette='Greens')
    ax3.set_title('Average P&L by Hour', color='#1B5E20')
    
    # Best vs Worst Trade
    times = pd.DataFrame({
        'Time': [best_trade[date_col].hour, worst_trade[date_col].hour],
        'P&L': [best_trade['Net P/L'], worst_trade['Net P/L']],
        'Type': ['Best', 'Worst']
    })
    sns.barplot(data=times, x='Type', y='P&L', ax=ax4, 
                palette=['#2E7D32', '#B71C1C'])
    ax4.set_title('Best vs Worst Trade', color='#1B5E20')
    
    plt.tight_layout()
    plt.savefig('trading_best_worst.png')
    plt.close()
    
    return True

def generate_html(metrics, plot_path):
    html_content = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Trading Performance</title>
        <style>
            body {{
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                margin: 0;
                padding: 20px;
                background-color: #f5f5f5;
            }}
            .container {{
                max-width: 1200px;
                margin: 0 auto;
            }}
            .header {{
                background-color: #2E7D32;
                color: white;
                padding: 20px;
                border-radius: 10px;
                margin-bottom: 20px;
            }}
            .metrics-grid {{
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
                gap: 20px;
                margin-bottom: 20px;
            }}
            .metric-card {{
                background-color: white;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                border-left: 4px solid #2E7D32;
            }}
            .metric-value {{
                font-size: 24px;
                font-weight: bold;
                color: #2E7D32;
            }}
            .metric-label {{
                color: #1B5E20;
                font-size: 14px;
            }}
            .charts-container {{
                background-color: white;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                margin-top: 20px;
            }}
            img {{
                max-width: 100%;
                height: auto;
                margin-bottom: 20px;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>Trading Performance Dashboard</h1>
            </div>
            
            <div class="metrics-grid">
                <div class="metric-card">
                    <div class="metric-value">${metrics['total_pnl']:,.2f}</div>
                    <div class="metric-label">Total P&L</div>
                </div>
                <div class="metric-card">
                    <div class="metric-value">{metrics['win_rate']:.1f}%</div>
                    <div class="metric-label">Win Rate</div>
                </div>
                <div class="metric-card">
                    <div class="metric-value">{metrics['total_trades']}</div>
                    <div class="metric-label">Total Trades</div>
                </div>
                <div class="metric-card">
                    <div class="metric-value">${metrics['avg_profit']:.2f}</div>
                    <div class="metric-label">Average Profit/Loss</div>
                </div>
                <div class="metric-card">
                    <div class="metric-value">{metrics['sharpe']:.2f}</div>
                    <div class="metric-label">Sharpe Ratio</div>
                </div>
                <div class="metric-card">
                    <div class="metric-value">{metrics['sortino']:.2f}</div>
                    <div class="metric-label">Sortino Ratio</div>
                </div>
            </div>
            
            <div class="charts-container">
                <img src="{plot_path}" alt="Trading Analytics">
                <img src="trading_best_worst.png" alt="Best/Worst Analysis">
            </div>
        </div>
    </body>
    </html>
    '''
    return html_content

def main():
    file_path = input("Enter the path to your trading data file (CSV or Excel): ")
    
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    else:
        df = pd.read_excel(file_path)
    
    print("\nAvailable columns in your data:")
    for col in df.columns:
        print(f"- {col}")
    
    date_col = detect_date_column(df)
    if not date_col:
        date_col = input("\nEnter the name of your date/time column: ")
    
    metrics = calculate_metrics(df, date_col)
    create_plots(df, date_col)
    
    html_content = generate_html(metrics, 'trading_analytics.png')
    html_path = 'trading_dashboard.html'
    
    with open(html_path, 'w') as f:
        f.write(html_content)
    
    webbrowser.open('file://' + os.path.realpath(html_path))

if __name__ == '__main__':
    main()


Available columns in your data:
- Account
- Date/Time
- Symbol
- Description
- Strike price
- Side
- Order type
- Quantity
- Price
- Gross P/L
- Fee
- Net P/L
- Trade value
- Connection name
- Exchange
- Unnamed: 15


c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\site-packages\seaborn\_oldcore.py:1119: F

version 2